In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *

In [3]:
from fastai.vision.all import *

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:
import pandas as pd
import numpy as np
import regex as re

In [6]:
from fastai.callback.fp16 import *

In [7]:
cuda0 = torch.device('cuda:0')
cpu = torch.device('cpu')

In [8]:
def get_y(r): return parent_label(r).split(" ")

In [15]:
from torch.nn.modules import loss

from fastai.losses import *

class smooth_binary_cross_entropy(loss._Loss):

    def __init__(self, weight: Optional[Tensor] = None, size_average=None, reduce=None, reduction: str = 'mean',
             pos_weight: Optional[Tensor] = None) -> None:
        super(smooth_binary_cross_entropy, self).__init__(size_average, reduce, reduction)
        self.register_buffer('weight', weight)
        self.register_buffer('pos_weight', pos_weight)
    # def __init__(self,
    #              pos_weight: Optional[Tensor] = None,
    #              reduction: str = 'mean',
    #              thresh=0.5) -> None:
    #     store_attr()
    #
    #     self.pos_weight = pos_weight
    #     self.reduction = reduction
    #     self.thresh = thresh

    # pos_weight = torch.ones(3, device=cuda0)

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        c = target.shape[1]
        eps = 0.1
        smoothed_target = torch.where(target==1, 1-(eps+(eps/c)), eps/c)
        return F.binary_cross_entropy_with_logits(input,
                                                  smoothed_target,
                                                  pos_weight=self.pos_weight,
                                                  reduction=self.reduction)

    # def activation(self, x): return torch.sigmoid(x)

@delegates()
class smooth_loss_v2(BaseLoss):
    "Same as `nn.BCEWithLogitsLoss`, but flattens input and target."
    @use_kwargs_dict(keep=True, weight=None, reduction='mean', pos_weight=None)
    def __init__(self, *args, axis=-1, floatify=True, thresh=0.5, **kwargs):
        if kwargs.get('pos_weight', None) is not None and kwargs.get('flatten', None) is True:
            raise ValueError("`flatten` must be False when using `pos_weight` to avoid a RuntimeError due to shape mismatch")
        if kwargs.get('pos_weight', None) is not None: kwargs['flatten'] = False
        super().__init__(smooth_binary_cross_entropy, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)
        self.thresh = thresh

    def decodes(self, x):    return x>self.thresh
    def activation(self, x): return torch.sigmoid(x)

In [10]:
path = Path("../data/test")

In [11]:
images = get_image_files(path)
images

(#610) [Path('../data/test/008FWT.JPG'),Path('../data/test/00AQXY.JPG'),Path('../data/test/01OJZX.JPG'),Path('../data/test/07OXKK.jfif'),Path('../data/test/085IEC.jpg'),Path('../data/test/08O2YE.JPG'),Path('../data/test/08WLJO.jfif'),Path('../data/test/0E1VTP.jfif'),Path('../data/test/0GJFRQ.JPG'),Path('../data/test/0J3PQ7.JPG')...]

In [12]:
images[0].name

'008FWT.JPG'

In [16]:
models_dir = Path('../models/exported')
learn_inf = load_learner(models_dir/"142537_resnet50_B64S460.pkl", cpu=False)

In [ ]:
def get_dls() -> DataBlock:
    dblock = DataBlock(blocks=(ImageBlock),
                       item_tfms=Resize(460))

    return dblock.dataloaders(images, num_workers=0)

dls = get_dls()

In [ ]:
# tfms = [PILImage.create]
# dsets = Datasets(images, tfms=tfms, train_setup=False)
# dls = dsets.dataloaders(after_item=[Resize(460), ToTensor(), IntToFloatTensor()])

In [17]:
test_dl = learn_inf.dls.test_dl(images)

In [ ]:
# tst = test_set(dsets, images)

In [18]:
with learn_inf.no_bar():
    predictions, _ = learn_inf.tta(dl=test_dl)

In [19]:
len(predictions)

610

In [20]:
image_ids = [image.name.split(".")[0] for image in images]
preds = [[leaf_rust.item(), stem_rust.item(), healthy_wheat.item()] for healthy_wheat, stem_rust, leaf_rust in predictions]
df = pd.DataFrame(preds, columns=('leaf_rust', 'stem_rust', 'healthy_wheat'), index=image_ids)
df.index.name = "ID"

In [21]:
df = df.round(decimals=2)

In [23]:
df.to_csv(Path("answer.csv"))

In [22]:
df

,leaf_rust,stem_rust,healthy_wheat
ID,,,
008FWT,0.14,0.84,0.27
00AQXY,0.89,0.15,0.24
01OJZX,0.23,0.69,0.28
07OXKK,0.20,0.37,0.73
085IEC,0.50,0.03,0.77
...,...,...,...
ZKF0L9,0.23,0.89,0.21
ZMAB3R,0.55,0.52,0.29
ZOXDKA,0.52,0.13,0.30
